# Звіт з лабораторної роботи №9

Виконав студент групи КН-24-1 Озівський В. В.

Тема: Алгоритми на рядках
\
Мета: освоїти низку основних алгоритмів на рядках засобами мови Python.

### <center>Хід роботи</center>

#### 1. Наївний пошук підрядка

In [9]:
import time
import matplotlib.pyplot as plt
import numpy as np

def naive_match(p, t):
    assert len(p) <= len(t), "Довжина шаблону не може бути більшою за довжину тексту"
    
    occurrences = []
    
    # Проходимо по всіх можливих позиціях в тексті
    for i in range(len(t) - len(p) + 1):
        match = True
        
        # Перевіряємо кожен символ шаблону
        for j in range(len(p)):
            if t[i + j] != p[j]:
                match = False
                break
        
        if match:
            occurrences.append(i)
    
    return occurrences

#### 2. Z-функція

In [4]:
def z_func(s):
    n = len(s)
    if n == 0:
        return []
    
    # Ініціалізуємо масив Z
    Z = [0] * n
    Z[0] = n  # Z[0] завжди дорівнює довжині рядка
    
    if n == 1:
        return Z
    
    # Обчислюємо Z[1]
    for i in range(1, n):
        if s[i] == s[i-1]:
            Z[1] += 1
        else:
            break
    
    # Ініціалізуємо l та r
    r, l = 0, 0
    if Z[1] > 0:
        r, l = Z[1], 1
    
    # Обчислюємо Z[k] для k = 2, 3, ..., n-1
    for k in range(2, n):
        if k > r:
            # Випадок 1: k поза межами [l, r]
            for i in range(k, n):
                if s[i] == s[i - k]:
                    Z[k] += 1
                else:
                    break
            if Z[k] > 0:
                r, l = k + Z[k] - 1, k
        else:
            # Випадок 2: k всередині [l, r]
            nbeta = r - k + 1
            Zkp = Z[k - l]
            
            if nbeta > Zkp:
                # Підвипадок 2а
                Z[k] = Zkp
            else:
                # Підвипадок 2б
                nmatch = 0
                for i in range(r + 1, n):
                    if s[i] == s[nbeta + nmatch]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                Z[k] = nbeta + nmatch
    
    return Z

#### 3. Пошук підрядка за допомогою Z-функції

In [5]:
def z_match(p, t):
    # Формуємо розширений рядок p$t
    s = p + "$" + t
    
    # Обчислюємо Z-функцію
    Z = z_func(s)
    
    occurrences = []
    
    # Шукаємо позиції, де Z[i] == len(p)
    for i in range(len(p) + 1, len(s)):
        if Z[i] == len(p):
            # Позиція в оригінальному тексті
            occurrences.append(i - len(p) - 1)
    
    return occurrences

#### 4. Стиснення рядка за допомогою Z-функції

In [6]:
def compress_with_z(s):
    if not s:
        return s
    
    z_vec = z_func(s)
    n = len(s)
    
    # Перевіряємо всі можливі довжини періоду
    for i in range(1, n):
        # Умова: i + z_vec[i] == n означає, що суфікс збігається з префіксом
        # Умова: n % i == 0 означає, що довжина рядка ділиться на довжину періоду
        if i + z_vec[i] == n and n % i == 0:
            return s[:i]
    
    # Якщо не знайдено періоду, повертаємо весь рядок
    return s

#### 5. Тестування алгоритмів

In [12]:
def test_algorithms():   
    print("=== Тестування алгоритмів на рядках ===\n")
    
    # Тестові дані
    test_cases = [
        ("ab", "ababcababa"),
        ("abc", "abcabcabcabc"),
        ("needle", "needleneedleneedle"),
        ("кіт", "кіт у мішку, а мішок на возі, кіт"),
        ("aba", "abababaababa"),
        ("xyz", "abcdefghijk")
    ]
    
    for pattern, text in test_cases:
        print(f"Шаблон: '{pattern}', Текст: '{text}'")
        print(f"Довжина тексту: {len(text)}, Довжина шаблону: {len(pattern)}")
        
        # Наївний пошук
        naive_result = naive_match(pattern, text)
        print(f"Наївний пошук: {naive_result}")
        
        # Пошук з Z-функцією
        z_result = z_match(pattern, text)
        print(f"Z-пошук: {z_result}")
        
        # Перевірка на збіг результатів
        if naive_result == z_result:
            print("Результати збігаються")
        else:
            print("Результати не збігаються!")
        
        print("-" * 50)
    
    # Тестування стиснення
    print("\n=== Тестіванн ястиснення рядків ===\n")
    
    compression_tests = [
        "abcabcabc",
        "aaaa",
        "abababab",
        "абирвалгабирвалгабирвалг",
        "abcdef",
        "121212121212"
    ]
    
    for s in compression_tests:
        compressed = compress_with_z(s)
        z_values = z_func(s)
        
        print(f"Рядок: '{s}'")
        print(f"Z-функція: {z_values}")
        print(f"Стиснення: '{compressed}'")
        print(f"Коефіцієнт стиснення: {len(s) // len(compressed) if compressed else 1}")
        print("-" * 40)

In [13]:
test_algorithms()

=== Тестування алгоритмів на рядках ===

Шаблон: 'ab', Текст: 'ababcababa'
Довжина тексту: 10, Довжина шаблону: 2
Наївний пошук: [0, 2, 5, 7]
Z-пошук: [0, 2, 5, 7]
Результати збігаються
--------------------------------------------------
Шаблон: 'abc', Текст: 'abcabcabcabc'
Довжина тексту: 12, Довжина шаблону: 3
Наївний пошук: [0, 3, 6, 9]
Z-пошук: [0, 3, 6, 9]
Результати збігаються
--------------------------------------------------
Шаблон: 'needle', Текст: 'needleneedleneedle'
Довжина тексту: 18, Довжина шаблону: 6
Наївний пошук: [0, 6, 12]
Z-пошук: [0, 6, 12]
Результати збігаються
--------------------------------------------------
Шаблон: 'кіт', Текст: 'кіт у мішку, а мішок на возі, кіт'
Довжина тексту: 33, Довжина шаблону: 3
Наївний пошук: [0, 30]
Z-пошук: [0, 30]
Результати збігаються
--------------------------------------------------
Шаблон: 'aba', Текст: 'abababaababa'
Довжина тексту: 12, Довжина шаблону: 3
Наївний пошук: [0, 2, 4, 7, 9]
Z-пошук: [0, 2, 4, 7, 9]
Результати збігают

#### 6. Асимптотична складність алгоритмів

1. Наївний алгоритм пошуку:
   - У найкращому випадку: $O(n)$ - коли шаблон не знайдено на першій позиції
   - У середньому випадку: $O(n)$ для випадкових рядків
   - У найгіршому випадку: $O(n*m)$, де n - довжина тексту, m - довжина шаблону
   - Приклад найгіршого випадку: шаблон 'aaa...ab', текст 'aaa...aaa...a'

2. Z-алгоритм:
   - Часова складність: $O(n + m)$ - лінійна від загальної довжини
   - Просторова складність: $O(n + m)$ - для зберігання Z-масиву

3. Стиснення рядка з Z-Функцією:
   - Часова складність: $O(n)$ - обчислення Z-функції $+ O(n)$ прохід по масиву
   - Загальна складність: $O(n)$
   - Просторова складність: $O(n)$ - для Z-масиву

### Відповіді на контрольні питання

1. Префікс-функція vs Z-функція
- Префікс-функція π[i] = довжина найдовшого власного префікса рядка s[0..i], який є також суфіксом цього підрядка.
- Z-функція Z[i] = довжина найдовшого префікса рядка s, який збігається з підрядком, що починається з позиції i.
- Різниця: префікс-функція працює з підрядками змінної довжини, Z-функція завжди порівнює з початком всього рядка.

2. Z-функція в контексті алгоритмів
\
Z-функція - масив, де Z[i] показує максимальну кількість символів з позиції i, що збігаються з початком рядка.

Роль:
- Пошук підрядка за O(n+m)
- Стиснення рядків (пошук періодів)
- Аналіз повторів у рядках
- Побудова суфіксних структур

3. Найдовший спільний підрядок (LCS)

Підходи:
- Динамічне програмування - O(n×m) час, O(n×m) пам'ять
- Оптимізована DP - O(n×m) час, O(min(n,m)) пам'ять
- Хешування + бінарний пошук - O((n+m)×log(min(n,m)))
- Суфіксні масиви - для множинних запитів

4. Застосування в NLP та обробці текстів

Практичні застосування:
- Пошукові системи - індексація та пошук документів
- Перевірка плагіату - знаходження схожих фрагментів
- Автодоповнення - пошук префіксів у словниках
- Машинний переклад - вирівнювання речень
- Spell-checker - виправлення помилок через відстань редагування
- Біоінформатика - аналіз ДНК послідовностей
- Стиснення тексту - знаходження повторюваних паттернів